In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('mol2vec.csv')
df.replace(to_replace='#NAME?',value=np.nan,inplace=True)
df.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
columns = df.columns
df = df.astype("float", errors = "ignore")
df = np.float32(df)
df[np.isnan(df)] = 0
df[np.isinf(df)] = 0
df=pd.DataFrame(df,columns=columns)
ncol = df.shape[1]-1
x = df.iloc[:,0:ncol]
y = df['active']
corr_matrix = x.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
x1 = x.drop(to_drop, axis=1)
x2 = x1.loc[:, (x1 != x1.iloc[0]).any()] 
x2 = x2.round(6)
import warnings
warnings.filterwarnings("ignore")

In [2]:
df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
0,-3.042170,-3.151222,-3.232676,7.560959,5.113686,-0.568981,-19.256578,-4.969994,16.522644,1.793195,...,12.655949,20.450172,4.086627,-10.276544,-2.190987,-1.115333,-9.388687,-15.132810,-4.057350,1.0
1,-4.727887,-4.351881,-4.658237,-1.792979,-0.499149,-8.724982,-8.886705,5.068110,-1.706909,-1.742328,...,14.233682,-0.425539,-0.464745,-10.994161,-12.039774,-12.723747,-4.175037,-11.076891,2.639213,1.0
2,-4.097823,-3.983348,-5.248144,-0.722321,-1.265023,-8.232691,-7.873246,5.745502,-0.900785,-1.008448,...,13.890897,-0.387179,0.128263,-11.509965,-10.279533,-11.952808,-3.907961,-10.316779,2.601157,1.0
3,-5.616269,-6.542370,-6.168895,-3.166161,1.509026,-11.890589,-10.365218,4.797748,-1.633804,-2.156420,...,16.483412,1.412684,-0.536551,-12.108947,-14.011013,-14.057915,-5.047316,-14.451323,1.707880,1.0
4,-4.396163,-5.507332,-5.355181,-2.798300,1.738860,-10.982598,-10.979731,4.270193,-1.071588,-2.923557,...,16.781738,-0.069494,-0.499126,-10.858052,-13.415128,-14.032432,-5.018421,-12.879906,1.733685,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019,-0.245050,0.000723,-1.600506,2.734911,1.502080,0.887623,-5.741633,0.586250,4.324068,-0.374940,...,4.085058,5.202066,2.300924,-5.023169,-0.517164,-1.643741,-2.033808,-4.831588,0.884858,0.0
2020,-1.848185,-1.777678,-1.303566,0.651581,1.617647,1.635964,-2.599049,-0.967981,0.539179,1.095203,...,1.162264,3.851595,-0.020075,-1.346415,-1.949291,-0.575638,0.865116,-2.355083,-1.106393,0.0
2021,0.009402,0.458835,2.047133,-0.661602,1.160643,-0.072311,-4.937521,-2.287562,4.538611,-0.010992,...,1.278592,3.704251,0.990562,-2.375479,0.759892,-2.489954,-2.155487,-2.489589,-1.650008,0.0
2022,-3.098453,-1.974883,-2.323134,-0.688243,8.347803,0.660212,-16.231108,-3.275765,4.571203,-5.276364,...,3.041915,10.691944,3.452311,-7.664165,-4.948164,-1.304520,-8.855995,-8.327632,-5.991015,0.0


In [3]:
from sklearn.model_selection import train_test_split
x_train,x_val_test,y_train,y_val_test = train_test_split(x2,y,test_size = 0.2,random_state = 1)
x_val, x_test, y_val, y_test = train_test_split(x_val_test,y_val_test,test_size = 0.5,random_state = 1)

In [4]:
train = pd.concat([x_train,y_train],axis=1)
train

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
49,3.122140,1.383768,-2.587106,6.268650,0.471754,1.584855,-8.537524,1.572455,4.949413,2.413143,...,9.423290,7.713300,-0.026614,-8.864502,-4.885668,-5.977385,-3.910194,-7.264268,-2.071628,1.0
1709,0.386812,-2.901240,-1.103863,6.023128,-1.262581,1.163101,-6.473138,-0.432373,4.089872,-0.257529,...,6.469048,7.189582,-1.734141,-1.890889,-2.815710,-2.928446,-1.081778,-5.808842,0.756842,0.0
2009,1.590288,-1.896567,-0.352843,0.747566,-0.307107,0.503169,-0.507342,0.675988,-0.384693,-0.170826,...,2.426400,-0.238214,-0.691268,-2.288158,-1.070164,-2.147755,-0.791801,-0.277220,-0.238422,0.0
1700,-0.304303,-1.240169,-2.995203,3.142186,1.108402,-0.237587,-4.619317,-0.369691,3.224486,1.830230,...,3.700971,3.937362,0.142481,-3.635253,-2.883518,-1.820910,-1.269494,-4.479260,-1.203333,0.0
1873,0.063424,0.466783,-1.024059,0.759275,0.769761,0.258263,-1.557908,0.430519,0.221012,-0.200198,...,0.440741,0.412327,0.552912,-0.804914,-0.384307,-0.772130,-1.695097,-0.902029,-1.390793,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,-0.018476,0.109086,-0.968752,2.798012,0.166995,1.046289,-4.412730,-0.202914,2.708178,0.814531,...,3.652305,4.235707,0.144486,-2.727822,0.282026,-1.596515,-1.140338,-3.719178,-0.995245,0.0
1096,0.530779,-0.869595,-2.355478,4.892498,-0.347179,-0.044497,-5.422323,0.197992,4.825524,2.360238,...,5.431138,5.912798,0.432446,-2.924152,-0.754156,-1.680279,-0.424357,-4.695874,0.704580,0.0
1932,1.130856,-2.067951,-2.716505,0.642175,0.761404,2.476528,-10.015460,-1.877943,2.699778,3.333394,...,9.357099,7.393549,1.441057,-7.981624,-2.097896,-6.553251,-8.464458,-5.824588,-6.275820,0.0
235,0.038810,-6.001946,-3.352908,10.121549,2.350290,1.281578,-17.290070,-2.544254,12.236835,0.714338,...,13.834150,19.178854,-1.228625,-8.847307,-2.421464,-2.823674,-7.503624,-14.133018,-3.650405,1.0


In [5]:
val = pd.concat([x_val,y_val],axis=1)
val

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
979,-0.924738,-3.630971,-0.941683,2.288410,0.979304,3.368458,-6.296511,-1.376926,0.691666,-4.330696,...,8.206991,2.423658,-2.514117,-6.368415,-6.289406,-3.261788,-4.375273,-5.743221,-2.720430,1.0
321,3.981668,-3.817917,-4.334194,7.526982,-1.810609,-0.756866,-12.118903,0.160410,12.061170,5.389447,...,10.947912,11.856549,0.002532,-7.845087,-1.017480,-3.567741,-1.466021,-12.979085,-1.920076,1.0
1370,0.479289,-0.641769,-2.176216,4.274634,-2.247520,0.539447,-7.117249,0.990460,4.487190,3.354622,...,6.822226,6.694404,-1.180713,-3.777622,-0.748470,-1.608993,-1.698827,-4.064430,0.318094,0.0
1960,1.047405,-2.341650,-1.926396,6.968916,-1.028796,0.419297,-6.579942,0.238457,5.864690,2.993691,...,6.488382,7.728010,-0.442494,-2.926478,-1.737147,-1.826578,-0.196512,-6.090684,0.274805,0.0
700,-5.838797,-3.119025,-6.731914,-3.677045,0.177746,-11.161913,-19.402281,4.915827,6.432807,-0.836187,...,24.527470,3.236561,2.817659,-18.568508,-18.282352,-15.109824,-8.275272,-21.649940,4.314770,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,-0.357230,-2.816410,-2.715032,3.003527,0.450371,1.722460,-4.583472,-0.627402,6.087699,1.868356,...,6.593622,8.426202,0.205493,-5.575394,-1.960239,-3.551331,-4.707480,-7.438581,-3.629342,0.0
1653,1.480806,0.095987,-2.182062,3.154430,1.149957,1.053005,-3.588453,-0.029715,1.703170,0.182078,...,3.231322,4.180483,-1.296432,-2.927768,-1.133965,-2.159557,-0.782912,-5.327051,-1.523063,0.0
259,-5.083542,-7.306548,-10.971926,6.682369,5.043254,-17.272648,-26.685524,6.936164,15.695168,2.824468,...,27.697664,15.680459,6.016224,-20.506634,-17.976883,-14.157748,-9.583898,-23.538788,0.705302,1.0
1086,-1.262141,-4.160543,-3.855410,-0.203082,1.575167,-6.785056,-10.266095,0.137899,0.391242,0.674918,...,10.562466,3.121550,-0.992921,-9.413797,-8.808965,-7.304659,-4.304948,-11.421482,-0.681130,0.0


In [27]:
test1 = pd.concat([x_test,y_test],axis=1)
test1

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,active
655,-0.965457,-4.256200,-3.171449,3.005992,0.758764,-5.186360,-14.476170,4.813882,-0.010394,0.229098,...,12.271746,6.928438,0.447831,-15.207429,-10.400124,-8.441025,-2.675781,-12.339452,1.313364,1.0
957,3.084370,-1.971624,-1.772805,5.323298,-3.314483,4.717834,-5.058694,1.735718,4.364004,6.647315,...,7.593396,13.390501,-0.734711,-4.382991,0.397785,-5.647056,-2.212486,-7.238034,-2.472134,1.0
1976,0.930817,0.573079,-2.802256,0.454817,-1.487874,0.694428,-7.036918,0.356073,2.249614,2.743060,...,5.585688,5.623418,-0.665200,-3.884865,-1.095558,-1.630851,-3.226555,-5.093956,-1.212208,0.0
292,3.912698,-3.398960,-4.706953,7.955660,0.534628,0.609698,-14.284689,-1.359125,7.610195,3.322568,...,12.106884,11.589707,-0.902216,-11.304275,-3.721880,-8.180736,-6.954284,-15.553990,-2.804318,1.0
1183,-1.315449,-1.206371,-2.282106,-1.345010,-1.349415,-2.035216,-3.813459,3.534329,-0.193448,-0.812797,...,5.169467,0.031101,-0.826021,-4.118060,-5.600006,-4.220957,-1.315605,-5.425556,2.504330,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,0.312340,-3.042980,-3.870216,3.758436,2.017219,-0.857866,-6.784304,1.361167,2.026633,-0.697731,...,6.413473,4.756366,-0.724946,-7.365251,-4.125722,-4.834089,-2.594676,-7.612018,-1.607808,1.0
177,0.341812,-1.535117,-5.777174,8.168260,3.429134,-0.319100,-13.904315,1.459436,5.696040,2.160142,...,12.126826,12.120297,-0.525164,-8.288666,-5.095169,-7.797640,-3.721126,-13.834191,-1.110383,1.0
1775,-3.367420,-1.359982,-3.106246,0.698338,4.552271,-2.146742,-8.062738,-0.512359,1.661504,-2.334532,...,5.052576,4.371771,1.462733,-4.878756,-3.090804,-2.195801,-2.562334,-6.103112,-3.038967,0.0
1426,-1.311868,-4.731458,-3.726913,0.098447,-1.668079,-8.189605,-9.332282,4.525275,-0.874134,-1.667287,...,14.271614,-1.991005,-1.691351,-9.341183,-9.904122,-11.220035,-2.435872,-10.329916,4.051055,0.0


In [7]:
col_name = train.columns.values.tolist()
col_name.pop()

'active'

In [8]:
from pycaret.classification import *
CM = setup(data = train, target = 'active', test_data = val, silent=True, numeric_features = col_name,
             session_id=0, fold_strategy = 'stratifiedkfold', fold = 5, fold_shuffle = True, normalize = True, normalize_method = 'minmax')

,Description,Value
0,session_id,0
1,Target,active
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(1619, 300)"
5,Missing Values,False
6,Numeric Features,299
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
mlp = create_model('mlp')
tunedmlp = tune_model(mlp,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedmlp)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8179,0.9024,0.9103,0.7416,0.8173,0.6395,0.6529
1,0.8302,0.8911,0.7655,0.8409,0.8014,0.6538,0.6560
2,0.8395,0.9014,0.8069,0.8298,0.8182,0.6746,0.6748
3,0.8704,0.9266,0.8897,0.8323,0.8600,0.7396,0.7410
4,0.7802,0.8598,0.7793,0.7434,0.7609,0.5577,0.5582
Mean,0.8276,0.8963,0.8303,0.7976,0.8116,0.6530,0.6566
Std,0.0294,0.0216,0.0588,0.0451,0.0319,0.0587,0.0585


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,MLP Classifier,0.8465,0.9101,0.8353,0.8068,0.8208,0.6867,0.6870


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.571995,0.794332,0.780089,0.377135,0.244132,0.757407,0.874537,0.744004,0.237695,0.428149,...,0.117648,0.803207,0.768610,0.848928,0.838030,0.913790,0.379076,1.0,0.0,0.9237
1,0.741740,0.789558,0.658841,0.555539,0.164746,0.646750,0.801727,0.785593,0.349382,0.755408,...,0.188885,0.774805,0.892881,0.841465,0.876390,0.828286,0.407228,1.0,1.0,0.5609
2,0.620569,0.870662,0.735967,0.444777,0.152314,0.681522,0.864273,0.808048,0.274980,0.686899,...,0.155392,0.853037,0.899222,0.889244,0.873321,0.933627,0.485954,0.0,0.0,0.9381
3,0.640224,0.827255,0.744895,0.536533,0.186992,0.678299,0.870992,0.787704,0.288512,0.674748,...,0.176288,0.869408,0.875916,0.883937,0.893130,0.909684,0.484431,0.0,0.0,0.8493
4,0.401985,0.807405,0.573147,0.173976,0.221324,0.367647,0.710648,0.914238,0.294093,0.545803,...,0.268570,0.568555,0.485911,0.559920,0.786606,0.725827,0.626535,1.0,1.0,0.8930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.591628,0.815132,0.716710,0.401488,0.229081,0.713255,0.895958,0.764281,0.290703,0.636860,...,0.194630,0.818459,0.870658,0.841865,0.833650,0.893756,0.347105,0.0,0.0,0.8715
198,0.655218,0.889501,0.735758,0.406628,0.248988,0.695298,0.908401,0.780449,0.247632,0.580086,...,0.152116,0.869383,0.890135,0.875814,0.885397,0.918707,0.421192,0.0,0.0,0.9384
199,0.428114,0.700475,0.421611,0.526775,0.359769,0.203734,0.619571,0.968893,0.385080,0.669050,...,0.359109,0.531277,0.493112,0.583144,0.769351,0.703507,0.499574,1.0,1.0,0.7599
200,0.560322,0.780809,0.675953,0.292285,0.261087,0.485051,0.824897,0.784984,0.234744,0.596679,...,0.160707,0.744633,0.709219,0.750310,0.838957,0.846692,0.450807,0.0,0.0,0.7590


In [10]:
xgb = create_model('xgboost')
tunedxgb = tune_model(xgb,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedxgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8488,0.9234,0.8138,0.8429,0.8281,0.6932,0.6935
1,0.8117,0.9113,0.8138,0.7763,0.7946,0.6210,0.6216
2,0.8426,0.9183,0.8276,0.8219,0.8247,0.6819,0.6819
3,0.8580,0.9462,0.8345,0.8462,0.8403,0.7125,0.7126
4,0.8019,0.8851,0.7862,0.7755,0.7808,0.6000,0.6001
Mean,0.8326,0.9169,0.8152,0.8126,0.8137,0.6617,0.6619
Std,0.0219,0.0197,0.0166,0.0310,0.0223,0.0434,0.0434


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8515,0.9208,0.8471,0.8090,0.8276,0.6973,0.6978


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.571995,0.794332,0.780089,0.377135,0.244132,0.757407,0.874537,0.744004,0.237695,0.428149,...,0.117648,0.803207,0.768610,0.848928,0.838030,0.913790,0.379076,1.0,0.0,0.9522
1,0.741740,0.789558,0.658841,0.555539,0.164746,0.646750,0.801727,0.785593,0.349382,0.755408,...,0.188885,0.774805,0.892881,0.841465,0.876390,0.828286,0.407228,1.0,1.0,0.7139
2,0.620569,0.870662,0.735967,0.444777,0.152314,0.681522,0.864273,0.808048,0.274980,0.686899,...,0.155392,0.853037,0.899222,0.889244,0.873321,0.933627,0.485954,0.0,0.0,0.9893
3,0.640224,0.827255,0.744895,0.536533,0.186992,0.678299,0.870992,0.787704,0.288512,0.674748,...,0.176288,0.869408,0.875916,0.883937,0.893130,0.909684,0.484431,0.0,0.0,0.9384
4,0.401985,0.807405,0.573147,0.173976,0.221324,0.367647,0.710648,0.914238,0.294093,0.545803,...,0.268570,0.568555,0.485911,0.559920,0.786606,0.725827,0.626535,1.0,1.0,0.9581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.591628,0.815132,0.716710,0.401488,0.229081,0.713255,0.895958,0.764281,0.290703,0.636860,...,0.194630,0.818459,0.870658,0.841865,0.833650,0.893756,0.347105,0.0,0.0,0.9339
198,0.655218,0.889501,0.735758,0.406628,0.248988,0.695298,0.908401,0.780449,0.247632,0.580086,...,0.152116,0.869383,0.890135,0.875814,0.885397,0.918707,0.421192,0.0,0.0,0.9571
199,0.428114,0.700475,0.421611,0.526775,0.359769,0.203734,0.619571,0.968893,0.385080,0.669050,...,0.359109,0.531277,0.493112,0.583144,0.769351,0.703507,0.499574,1.0,1.0,0.8153
200,0.560322,0.780809,0.675953,0.292285,0.261087,0.485051,0.824897,0.784984,0.234744,0.596679,...,0.160707,0.744633,0.709219,0.750310,0.838957,0.846692,0.450807,0.0,0.0,0.8952


In [11]:
rf = create_model('rf')
tunedrf = tune_model(rf, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedrf)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8426,0.9220,0.8069,0.8357,0.8211,0.6806,0.6810
1,0.8179,0.9114,0.8138,0.7867,0.8000,0.6329,0.6333
2,0.8395,0.9190,0.8276,0.8163,0.8219,0.6759,0.6759
3,0.8580,0.9401,0.8483,0.8367,0.8425,0.7133,0.7133
4,0.7926,0.8865,0.8069,0.7500,0.7774,0.5837,0.5851
Mean,0.8301,0.9158,0.8207,0.8051,0.8126,0.6573,0.6577
Std,0.0227,0.0174,0.0157,0.0330,0.0221,0.0448,0.0444


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8416,0.9169,0.8471,0.7912,0.8182,0.6781,0.6794


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.571995,0.794332,0.780089,0.377135,0.244132,0.757407,0.874537,0.744004,0.237695,0.428149,...,0.117648,0.803207,0.768610,0.848928,0.838030,0.913790,0.379076,1.0,0.0,0.7768
1,0.741740,0.789558,0.658841,0.555539,0.164746,0.646750,0.801727,0.785593,0.349382,0.755408,...,0.188885,0.774805,0.892881,0.841465,0.876390,0.828286,0.407228,1.0,1.0,0.6354
2,0.620569,0.870662,0.735967,0.444777,0.152314,0.681522,0.864273,0.808048,0.274980,0.686899,...,0.155392,0.853037,0.899222,0.889244,0.873321,0.933627,0.485954,0.0,0.0,0.9800
3,0.640224,0.827255,0.744895,0.536533,0.186992,0.678299,0.870992,0.787704,0.288512,0.674748,...,0.176288,0.869408,0.875916,0.883937,0.893130,0.909684,0.484431,0.0,0.0,0.7781
4,0.401985,0.807405,0.573147,0.173976,0.221324,0.367647,0.710648,0.914238,0.294093,0.545803,...,0.268570,0.568555,0.485911,0.559920,0.786606,0.725827,0.626535,1.0,1.0,0.9428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.591628,0.815132,0.716710,0.401488,0.229081,0.713255,0.895958,0.764281,0.290703,0.636860,...,0.194630,0.818459,0.870658,0.841865,0.833650,0.893756,0.347105,0.0,0.0,0.7399
198,0.655218,0.889501,0.735758,0.406628,0.248988,0.695298,0.908401,0.780449,0.247632,0.580086,...,0.152116,0.869383,0.890135,0.875814,0.885397,0.918707,0.421192,0.0,0.0,0.8663
199,0.428114,0.700475,0.421611,0.526775,0.359769,0.203734,0.619571,0.968893,0.385080,0.669050,...,0.359109,0.531277,0.493112,0.583144,0.769351,0.703507,0.499574,1.0,1.0,0.7001
200,0.560322,0.780809,0.675953,0.292285,0.261087,0.485051,0.824897,0.784984,0.234744,0.596679,...,0.160707,0.744633,0.709219,0.750310,0.838957,0.846692,0.450807,0.0,0.0,0.7814


In [12]:
et = create_model('et')
tunedet = tune_model(et, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedet)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9182,0.8483,0.8255,0.8367,0.7012,0.7014
1,0.8210,0.9111,0.8276,0.7843,0.8054,0.6399,0.6407
2,0.8519,0.9207,0.8552,0.8212,0.8378,0.7016,0.7021
3,0.8673,0.9400,0.8552,0.8493,0.8522,0.7318,0.7318
4,0.8019,0.8894,0.8138,0.7613,0.7867,0.6021,0.6032
Mean,0.8388,0.9159,0.8400,0.8083,0.8238,0.6753,0.6758
Std,0.0238,0.0164,0.0166,0.0314,0.0240,0.0473,0.0469


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extra Trees Classifier,0.8317,0.9225,0.8353,0.7802,0.8068,0.6580,0.6592


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.571995,0.794332,0.780089,0.377135,0.244132,0.757407,0.874537,0.744004,0.237695,0.428149,...,0.117648,0.803207,0.768610,0.848928,0.838030,0.913790,0.379076,1.0,0.0,0.7935
1,0.741740,0.789558,0.658841,0.555539,0.164746,0.646750,0.801727,0.785593,0.349382,0.755408,...,0.188885,0.774805,0.892881,0.841465,0.876390,0.828286,0.407228,1.0,1.0,0.6667
2,0.620569,0.870662,0.735967,0.444777,0.152314,0.681522,0.864273,0.808048,0.274980,0.686899,...,0.155392,0.853037,0.899222,0.889244,0.873321,0.933627,0.485954,0.0,0.0,0.9628
3,0.640224,0.827255,0.744895,0.536533,0.186992,0.678299,0.870992,0.787704,0.288512,0.674748,...,0.176288,0.869408,0.875916,0.883937,0.893130,0.909684,0.484431,0.0,0.0,0.7560
4,0.401985,0.807405,0.573147,0.173976,0.221324,0.367647,0.710648,0.914238,0.294093,0.545803,...,0.268570,0.568555,0.485911,0.559920,0.786606,0.725827,0.626535,1.0,1.0,0.9418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.591628,0.815132,0.716710,0.401488,0.229081,0.713255,0.895958,0.764281,0.290703,0.636860,...,0.194630,0.818459,0.870658,0.841865,0.833650,0.893756,0.347105,0.0,0.0,0.7224
198,0.655218,0.889501,0.735758,0.406628,0.248988,0.695298,0.908401,0.780449,0.247632,0.580086,...,0.152116,0.869383,0.890135,0.875814,0.885397,0.918707,0.421192,0.0,0.0,0.8916
199,0.428114,0.700475,0.421611,0.526775,0.359769,0.203734,0.619571,0.968893,0.385080,0.669050,...,0.359109,0.531277,0.493112,0.583144,0.769351,0.703507,0.499574,1.0,1.0,0.6877
200,0.560322,0.780809,0.675953,0.292285,0.261087,0.485051,0.824897,0.784984,0.234744,0.596679,...,0.160707,0.744633,0.709219,0.750310,0.838957,0.846692,0.450807,0.0,0.0,0.7337


In [13]:
catboost = create_model('catboost')
tunedcat1 = tune_model(catboost, n_iter = 200,search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedcat1)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8426,0.9251,0.8138,0.8310,0.8223,0.6811,0.6812
1,0.8148,0.9152,0.8069,0.7852,0.7959,0.6265,0.6267
2,0.8426,0.9168,0.8138,0.8310,0.8223,0.6811,0.6812
3,0.8549,0.9404,0.8345,0.8403,0.8374,0.7065,0.7065
4,0.8019,0.8858,0.8069,0.7647,0.7852,0.6016,0.6023
Mean,0.8314,0.9167,0.8152,0.8104,0.8126,0.6593,0.6596
Std,0.0198,0.0178,0.0101,0.0299,0.0191,0.0389,0.0387


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,CatBoost Classifier,0.8614,0.9255,0.8588,0.8202,0.8391,0.7175,0.7180


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.571995,0.794332,0.780089,0.377135,0.244132,0.757407,0.874537,0.744004,0.237695,0.428149,...,0.117648,0.803207,0.768610,0.848928,0.838030,0.913790,0.379076,1.0,0.0,0.9192
1,0.741740,0.789558,0.658841,0.555539,0.164746,0.646750,0.801727,0.785593,0.349382,0.755408,...,0.188885,0.774805,0.892881,0.841465,0.876390,0.828286,0.407228,1.0,1.0,0.7119
2,0.620569,0.870662,0.735967,0.444777,0.152314,0.681522,0.864273,0.808048,0.274980,0.686899,...,0.155392,0.853037,0.899222,0.889244,0.873321,0.933627,0.485954,0.0,0.0,0.9709
3,0.640224,0.827255,0.744895,0.536533,0.186992,0.678299,0.870992,0.787704,0.288512,0.674748,...,0.176288,0.869408,0.875916,0.883937,0.893130,0.909684,0.484431,0.0,0.0,0.8902
4,0.401985,0.807405,0.573147,0.173976,0.221324,0.367647,0.710648,0.914238,0.294093,0.545803,...,0.268570,0.568555,0.485911,0.559920,0.786606,0.725827,0.626535,1.0,1.0,0.9811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.591628,0.815132,0.716710,0.401488,0.229081,0.713255,0.895958,0.764281,0.290703,0.636860,...,0.194630,0.818459,0.870658,0.841865,0.833650,0.893756,0.347105,0.0,0.0,0.8555
198,0.655218,0.889501,0.735758,0.406628,0.248988,0.695298,0.908401,0.780449,0.247632,0.580086,...,0.152116,0.869383,0.890135,0.875814,0.885397,0.918707,0.421192,0.0,0.0,0.9507
199,0.428114,0.700475,0.421611,0.526775,0.359769,0.203734,0.619571,0.968893,0.385080,0.669050,...,0.359109,0.531277,0.493112,0.583144,0.769351,0.703507,0.499574,1.0,1.0,0.7805
200,0.560322,0.780809,0.675953,0.292285,0.261087,0.485051,0.824897,0.784984,0.234744,0.596679,...,0.160707,0.744633,0.709219,0.750310,0.838957,0.846692,0.450807,0.0,0.0,0.8671


In [14]:
lightgbm = create_model('lightgbm')
tunedlightgbm = tune_model(lightgbm,n_iter = 200, search_library = 'scikit-optimize',optimize = 'AUC')
predict_model(tunedlightgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8519,0.9252,0.8207,0.8440,0.8322,0.6996,0.6998
1,0.8086,0.9106,0.8207,0.7677,0.7933,0.6155,0.6167
2,0.8364,0.9154,0.8138,0.8194,0.8166,0.6690,0.6690
3,0.8580,0.9412,0.8276,0.8511,0.8392,0.7121,0.7124
4,0.8080,0.8880,0.7793,0.7902,0.7847,0.6116,0.6116
Mean,0.8326,0.9161,0.8124,0.8145,0.8132,0.6616,0.6619
Std,0.0210,0.0175,0.0171,0.0316,0.0212,0.0417,0.0415


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8465,0.9210,0.8353,0.8068,0.8208,0.6867,0.6870


,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,active,Label,Score
0,0.571995,0.794332,0.780089,0.377135,0.244132,0.757407,0.874537,0.744004,0.237695,0.428149,...,0.117648,0.803207,0.768610,0.848928,0.838030,0.913790,0.379076,1.0,0.0,0.9554
1,0.741740,0.789558,0.658841,0.555539,0.164746,0.646750,0.801727,0.785593,0.349382,0.755408,...,0.188885,0.774805,0.892881,0.841465,0.876390,0.828286,0.407228,1.0,1.0,0.5252
2,0.620569,0.870662,0.735967,0.444777,0.152314,0.681522,0.864273,0.808048,0.274980,0.686899,...,0.155392,0.853037,0.899222,0.889244,0.873321,0.933627,0.485954,0.0,0.0,0.9979
3,0.640224,0.827255,0.744895,0.536533,0.186992,0.678299,0.870992,0.787704,0.288512,0.674748,...,0.176288,0.869408,0.875916,0.883937,0.893130,0.909684,0.484431,0.0,0.0,0.9563
4,0.401985,0.807405,0.573147,0.173976,0.221324,0.367647,0.710648,0.914238,0.294093,0.545803,...,0.268570,0.568555,0.485911,0.559920,0.786606,0.725827,0.626535,1.0,1.0,0.9881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,0.591628,0.815132,0.716710,0.401488,0.229081,0.713255,0.895958,0.764281,0.290703,0.636860,...,0.194630,0.818459,0.870658,0.841865,0.833650,0.893756,0.347105,0.0,0.0,0.9386
198,0.655218,0.889501,0.735758,0.406628,0.248988,0.695298,0.908401,0.780449,0.247632,0.580086,...,0.152116,0.869383,0.890135,0.875814,0.885397,0.918707,0.421192,0.0,0.0,0.9898
199,0.428114,0.700475,0.421611,0.526775,0.359769,0.203734,0.619571,0.968893,0.385080,0.669050,...,0.359109,0.531277,0.493112,0.583144,0.769351,0.703507,0.499574,1.0,1.0,0.8872
200,0.560322,0.780809,0.675953,0.292285,0.261087,0.485051,0.824897,0.784984,0.234744,0.596679,...,0.160707,0.744633,0.709219,0.750310,0.838957,0.846692,0.450807,0.0,0.0,0.9601


In [25]:
from itertools import combinations
ml = [tunedrf, tunedet, tunedlightgbm, tunedcat1, tunedxgb,tunedmlp]
a2 =  list(combinations(ml, 2))
a3 =  list(combinations(ml, 3))
a4 =  list(combinations(ml, 4))
a5 =  list(combinations(ml, 5))
a6 =  list(combinations(ml, 6))
all = a2 + a3 + a4 + a5 + a6 
print(len(all))
print(len(a2),len(a3),len(a4),len(a5),len(a6))

57
15 20 15 6 1


In [16]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['Accuracy']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8488,0.9213,0.8069,0.8478,0.8269,0.6928,0.6934
1,0.8117,0.9079,0.8000,0.7838,0.7918,0.6200,0.6201
2,0.8395,0.9173,0.8138,0.8252,0.8194,0.6750,0.6751
3,0.8642,0.9395,0.8483,0.8483,0.8483,0.7254,0.7254
4,0.7864,0.8833,0.7793,0.7533,0.7661,0.5696,0.5699
Mean,0.8301,0.9139,0.8097,0.8117,0.8105,0.6566,0.6568
Std,0.0277,0.0184,0.0225,0.0374,0.0286,0.0553,0.0553


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8366,0.9208,0.8353,0.7889,0.8114,0.6675,0.6684


(RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False), <catboost.core.CatBoostClassifier object at 0x7fa6749c33a0>, MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=0,

In [26]:
base_score = 0
for i in all:
    blender = blend_models(i)
    predict_model(blender)
    test = pull()
    avg = test['AUC']
    avg = avg.iloc[0]
    if avg > base_score:
       base_score = avg
       best_params = i
print(best_params, base_score)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8457,0.9233,0.8414,0.8188,0.8299,0.6887,0.6890
1,0.8241,0.9127,0.8207,0.7933,0.8068,0.6454,0.6457
2,0.8395,0.9182,0.8138,0.8252,0.8194,0.6750,0.6751
3,0.8673,0.9434,0.8552,0.8493,0.8522,0.7318,0.7318
4,0.7926,0.8892,0.7931,0.7566,0.7744,0.5826,0.5832
Mean,0.8338,0.9173,0.8248,0.8086,0.8166,0.6647,0.6649
Std,0.0249,0.0175,0.0216,0.0315,0.0258,0.0496,0.0494


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8416,0.9233,0.8471,0.7912,0.8182,0.6781,0.6794


(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced',
                     criterion='entropy', max_depth=11,
                     max_features=0.8565622090600215, max_leaf_nodes=None,
                     max_samples=None,
                     min_impurity_decrease=7.744950961828116e-09,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=9, min_weight_fraction_leaf=0.0,
                     n_estimators=300, n_jobs=-1, oob_score=False,
                     random_state=0, verbose=0, warm_start=False), <catboost.core.CatBoostClassifier object at 0x7fa6746c8b50>, XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.9360902645687517, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              i

In [28]:
blend = blend_models([tunedet,tunedcat1,tunedxgb])
md = predict_model(blend,data = test1)
md
y_pred = md['Label']
y_true = md['active']
t = y_pred.astype(float)
t
from sklearn.metrics import balanced_accuracy_score
accuracy = balanced_accuracy_score(y_true, t)
accuracy

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8426,0.9229,0.8138,0.8310,0.8223,0.6811,0.6812
1,0.8272,0.9137,0.8276,0.7947,0.8108,0.6518,0.6523
2,0.8457,0.9201,0.8345,0.8231,0.8288,0.6883,0.6884
3,0.8673,0.9435,0.8552,0.8493,0.8522,0.7318,0.7318
4,0.8019,0.8891,0.8069,0.7647,0.7852,0.6016,0.6023
Mean,0.8369,0.9179,0.8276,0.8126,0.8199,0.6709,0.6712
Std,0.0217,0.0175,0.0169,0.0297,0.0220,0.0431,0.0428


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.8966,0.9535,0.9022,0.8737,0.8877,0.7919,0.7922


0.8970329024676851